In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import preprocess as prep
import os
import shutil

In [2]:
def corner_point(position):
    x1, y1, x2, y2 = position
    return int(min(x1, x2)), int(min(y1, y2))

In [3]:
def data_refresh(data, x_final, y_final, window_shape_value):
    data_refresh = data[y_final:y_final+window_shape_value, x_final:x_final+window_shape_value]
    return data_refresh

In [4]:
# Save the data and asteroid position into a npz file
def save_data_negative(name, index, count, data, data_dir):
    dataPath = '{}\{}_{}'.format(data_dir, count, name)
    np.savez(dataPath, matrix1=data)
    print('Successfully save data for index:{} count:{} filename:{}'.format(index, count, name))

In [5]:
def plot_data_negative(name, count, data, pic_dir): 
    plt.imshow(data, cmap='gray', vmin=0, vmax=0.01)
    plt.colorbar()
    plt.savefig('{}\{}_{}.jpg'.format(pic_dir, count, name), dpi=300)
    plt.clf()

In [6]:
def generate_negative_data(father_dir, pic_dir, data_dir, start_index, end_index, window_shape_value=1000):
    count = 0
    
    for index in range(start_index, end_index):
        path, name = prep.path_of_data(father_dir, index)
        print('Start preprocessing {}: {}'.format(index, name))
        data, position = prep.load_saved_data(path)
        data = np.nan_to_num(data)
        x_corner, y_corner = corner_point(position)
        if x_corner > window_shape_value+500 and y_corner > window_shape_value+500:
            for i in range(2):
                x_final = random.randint(0, x_corner - window_shape_value)
                y_final = random.randint(0, y_corner - window_shape_value)
                data_new = data_refresh(data, x_final, y_final, window_shape_value)
                save_data_negative(name, index, count, data_new, data_dir)
                plot_data_negative(name, count, data_new, pic_dir)
                count += 1

In [7]:
def random_split(dir_train, dir_val):
    fileList = os.listdir(dir_train)
    for file in fileList:
        train_file = os.path.join(dir_train, file)
        val_file = os.path.join(dir_val, file)
        if random.random() < 0.2:
            shutil.move(train_file, val_file)
            print('File move from {} to {}'.format(train_file, val_file))
        else:
            print('File does not move')

In [9]:
if __name__ == '__main__':
    generate_negative_data('.\data', '.\pic_negative', '.\\preprocess\\train\\negative', 150, 200, 1000)
    random_split('.\\preprocess\\train\\negative', '.\\preprocess\\val\\negative')

Start preprocessing 150: 233-icxm02020
Start preprocessing 151: 234-icxm02030
Start preprocessing 152: 235-icxv21030
Start preprocessing 153: 236-icz903030
Successfully save data for index:153 count:0 filename:236-icz903030
Successfully save data for index:153 count:1 filename:236-icz903030
Start preprocessing 154: 237-iczgn1030
Start preprocessing 155: 238-iczgn1030
Start preprocessing 156: 239-iczgw1030
Start preprocessing 157: 24-ibi705010
Start preprocessing 158: 240-id1k5z010
Start preprocessing 159: 241-id1k5z010
Start preprocessing 160: 242-id1k65010
Start preprocessing 161: 243-id1k6a010
Successfully save data for index:161 count:2 filename:243-id1k6a010
Successfully save data for index:161 count:3 filename:243-id1k6a010
Start preprocessing 162: 244-id1k6a010
Start preprocessing 163: 245-id1k6a010
Successfully save data for index:163 count:4 filename:245-id1k6a010
Successfully save data for index:163 count:5 filename:245-id1k6a010
Start preprocessing 164: 246-id1k6b010
Start pr

<Figure size 640x480 with 0 Axes>